#Instalation of dependencies

In [1]:
%tensorflow_version 1.x
# Copy Mask RCNN, weights and dataset
!cp -r /content/drive/MyDrive/Mask_RCNN /content/Mask_RCNN/
!cp -r /content/drive/MyDrive/Bc-proj/logs /content/logs
!cp -r /content/drive/MyDrive/Bc-proj/dataset /content
%cd /content/Mask_RCNN
!pip install -r requirements.txt
!pip install h5py==2.10.0
!python setup.py install

TensorFlow 1.x selected.
/content/Mask_RCNN
     |████████████████████████████████| 163kB 5.7MB/s 
     |████████████████████████████████| 204kB 15.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=374f0fd4074e2f0c9b053dfe7553056f0d0659337a55b31888e8c0f75cdac00e
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.15.2 which is incompatible.
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
     |████████████████████████████████| 2.9MB 4.2MB/s 
  Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the 

In [2]:
import os


import sys
import random
import math
import json
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import cv2 as cv

ROOT_DIR = os.path.abspath('..')
sys.path.append(ROOT_DIR)

from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.config import Config
from mrcnn.model import log
from mrcnn.utils import Dataset

Using TensorFlow backend.


# Configuration of model

In [3]:
MODEL_DIR = os.path.abspath("/content/logs")
# Local path to trained weights file

# !cp /content/drive/MyDrive/Bc-proj/mask_rcnn_coco.h5 .

# COCO_MODEL_PATH = os.path.join('.', "mask_rcnn_coco.h5")

class McConfig(Config):
    # Give the configuration a recognizable name
    NAME = "Mc"

    # Batch size = (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + 1 bone

    IMAGE_MIN_DIM = 600
    IMAGE_MAX_DIM = 1600

    # RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    
    # Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 6

    
    STEPS_PER_EPOCH = 150

    
    VALIDATION_STEPS = 10


class InterfaceMc(McConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
c = InterfaceMc()
c.display

<bound method Config.display of <__main__.InterfaceMc object at 0x7fb288ee7290>>

# Creat model and load weights

In [4]:
!cp -r /content/drive/MyDrive/Bc-proj/logs /content/logs
MODEL_DIR = '/content/logs'

config = InterfaceMc()
model = modellib.MaskRCNN(mode="inference", config=config,
                                  model_dir=MODEL_DIR)
path = '/content/mask_rcnn_mc.h5'
model.load_weights(path, by_name=True)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.
Re-starting from epoch 300


# Detecting masks of files

In [7]:
import cv2 as cv
%matplotlib inline
def get_files(dir: str):
  """Creates and returns list of files in directory dir."""
  files = []
  for path, dirs, fs in os.walk(dir):
            files += [os.path.join(path, f) for f in fs if '.tif' in f]
  return files

def load_img(img_path):
  if not os.path.isfile(img_path):
    raise ValueError(f'path: \'{img_path}\' is not valid file path.')
  else:
    img = cv.imread(img_path)
    if img is None:
      raise ValueError(f'Error while reading image \'{img_path}\'')
    else: return img
  return None

def get_mask(model, img):
  masks = model.detect([img], verbose=1)
  if masks:
    mask = masks[0]['masks']
    return mask
  return None

def create_binary_img(mask, img):
  result = np.zeros(img.shape)
  result[:] = 255
  #mask = (np.sum(mask, -1, keepdims=True) >= 1)
  r = np.reshape(mask, img.shape[:2])
  result[r] = np.array([0, 0, 0], dtype=np.uint8)
  return result





In [ ]:
!mkdir -p /content/output
OUT_DIR = '/content/output/'
files = get_files('/content/dataset/')
for f in files:
  img = load_img(f)
  mask = get_mask(model, img)
  out_f_path = os.path.join(OUT_DIR, f.split('/')[-1].split('.')[0] + '.npy')
  np.save(out_f_path, mask)
  
!tar -cavf /content/out.tgz /content/output/
